In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
# !wget http://parl.ai/downloads/projects/rephrase_sentences/rephrase_sentences_train_0703.txt
# !wget http://parl.ai/downloads/projects/rephrase_sentences/rephrase_sentences_valid_0703.txt
# !wget http://parl.ai/downloads/projects/rephrase_sentences/rephrase_sentences_test_0703.txt

In [14]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/paraphrase/translated-paraphrase.json

In [4]:
fnames = [
        'rephrase_sentences_train_0703.txt',
        'rephrase_sentences_valid_0703.txt',
        'rephrase_sentences_test_0703.txt',]

In [5]:
def _strip_reader(filename):
    """
    Reads a file, stripping line endings.
    """
    with open(filename) as f:
        for line in f:
            yield line.rstrip()

def grouper(iterable, n, fillvalue=None):
    """
    Collect data into fixed-length chunks or blocks.
    """
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    from itertools import zip_longest

    return zip_longest(*args, fillvalue=fillvalue)

In [6]:
data_reader = grouper(_strip_reader('rephrase_sentences_train_0703.txt'), 3, '')

In [15]:
entries = []

title_prefix = '1 passage title: '
title_prefix_len = len(title_prefix)
persona_prefix = '2 personality: '
persona_prefix_len = len(persona_prefix)
text_prefix = '3 '
text_prefix_len = len(text_prefix)

file = fnames[0]
data_reader = grouper(_strip_reader(file), 3, '')

for title, persona, text in data_reader:
    if not title:
        break
    assert title.startswith(title_prefix)
    title = title[title_prefix_len:]

    assert persona.startswith(persona_prefix)
    # strip 'persona: ' from the persona
    persona = persona[persona_prefix_len:]

    assert text.startswith(text_prefix)
    text = text[text_prefix_len:]

    passage, label = text.split("\t")

    entries.append(
        {'title': title, 'label': label, 'passage': passage, 'persona': persona}
    )
    
len(entries)

81467

In [16]:
entries[0]

{'title': 'Danio',
 'label': "Well, I bet you didn't know about danio's two pairs of long barbels",
 'passage': 'Some species have two pairs of long barbels.',
 'persona': 'Cultured (Refined, Educated)'}

In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')
model = T5ForConditionalGeneration.from_pretrained('mesolitica/finetune-translation-t5-small-standard-bahasa-cased')

In [18]:
_ = model.cuda()

In [ ]:
from tqdm import tqdm

for i in tqdm(range(len(entries))):
    batch = [entries[i]['label'], entries[i]['passage']]
    t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
    inputs = tokenizer(t, return_tensors="pt", padding = True)
    for k in inputs.keys():
        inputs[k] = inputs[k].cuda()

    translated_tokens = model.generate(**inputs, max_length=500)
    decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    entries[i]['ms'] = decoded

 69%|████████████████████████▏          | 56196/81467 [1:19:04<34:56, 12.05it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [31]:
with open('funpedia-train.json', 'w') as fopen:
    json.dump(entries, fopen)

In [35]:
entries = []
file = fnames[1]
data_reader = grouper(_strip_reader(file), 3, '')

for title, persona, text in data_reader:
    if not title:
        break
    assert title.startswith(title_prefix)
    title = title[title_prefix_len:]

    assert persona.startswith(persona_prefix)
    # strip 'persona: ' from the persona
    persona = persona[persona_prefix_len:]

    assert text.startswith(text_prefix)
    text = text[text_prefix_len:]

    passage, label = text.split("\t")

    entries.append(
        {'title': title, 'label': label, 'passage': passage, 'persona': persona}
    )
    
len(entries)

10184

In [36]:
for i in tqdm(range(len(entries))):
    batch = [entries[i]['label'], entries[i]['passage']]
    t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
    inputs = tokenizer(t, return_tensors="pt", padding = True)
    for k in inputs.keys():
        inputs[k] = inputs[k].cuda()

    translated_tokens = model.generate(**inputs, max_length=500)
    decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    entries[i]['ms'] = decoded

100%|█████████████████████████████████████| 10184/10184 [13:32<00:00, 12.54it/s]


In [37]:
with open('funpedia-valid.json', 'w') as fopen:
    json.dump(entries, fopen)

In [38]:
entries = []
file = fnames[2]
data_reader = grouper(_strip_reader(file), 3, '')

for title, persona, text in data_reader:
    if not title:
        break
    assert title.startswith(title_prefix)
    title = title[title_prefix_len:]

    assert persona.startswith(persona_prefix)
    # strip 'persona: ' from the persona
    persona = persona[persona_prefix_len:]

    assert text.startswith(text_prefix)
    text = text[text_prefix_len:]

    passage, label = text.split("\t")

    entries.append(
        {'title': title, 'label': label, 'passage': passage, 'persona': persona}
    )
    
len(entries)

10184

In [39]:
for i in tqdm(range(len(entries))):
    batch = [entries[i]['label'], entries[i]['passage']]
    t = ['terjemah Inggeris ke Melayu: ' + b for b in batch]
    inputs = tokenizer(t, return_tensors="pt", padding = True)
    for k in inputs.keys():
        inputs[k] = inputs[k].cuda()

    translated_tokens = model.generate(**inputs, max_length=500)
    decoded = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    entries[i]['ms'] = decoded

100%|█████████████████████████████████████| 10184/10184 [14:21<00:00, 11.82it/s]


In [40]:
with open('funpedia-test.json', 'w') as fopen:
    json.dump(entries, fopen)